<a href="https://colab.research.google.com/github/Mueble2/NLP-ChatBot-TA/blob/main/ChatBot_TA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

🤖 **Chatbot Académico: Historia del Perú**

In [1]:
!pip install sentence-transformers chromadb langchain langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 22.1 MB/s eta

In [2]:
# 📡 Carga de contenido web
import requests
from bs4 import BeautifulSoup
import re
# 🧠 Modelo de embedding (sentence transformers)
from sentence_transformers import SentenceTransformer
# 🧠 Embeddings y Vector Store
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from chromadb.config import Settings
import chromadb
# 📄 Procesamiento de documentos
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
# 🔍 Motor de recuperación de información
from langchain.chains import RetrievalQA
# 🤖 LLM vía Ollama
from langchain_community.llms import Ollama
#Para cargar documentos
from langchain_community.document_loaders import PyPDFLoader


In [3]:
def extraer_texto(url):
    try:
        resp = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=10)
        resp.raise_for_status()  # Lanza excepción si falla (4xx/5xx)
    except requests.RequestException as e:
        print(f"❌ Error al acceder a {url}: {e}")
        return ""
    soup = BeautifulSoup(resp.text, "html.parser")
    # Elimina etiquetas no deseadas
    for tag in soup(["script", "style", "sup", "img", "figure", "table", "noscript"]):
        tag.decompose()
    # Extrae texto limpio
    texto = soup.get_text(separator=' ')
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()

urls = [
    "https://es.wikipedia.org/wiki/Batalla_de_Ayacucho",
    "https://en.wikipedia.org/wiki/Battle_of_Ayacucho",
    "https://www.encyclopedia.com/humanities/encyclopedias-almanacs-transcripts-and-maps/ayacucho-battle",
    "https://www.britannica.com/topic/Battle-of-Ayacucho",
    "https://www.britannica.com/place/Ayacucho-Peru",
    "https://www.tierrasvivas.com/en/ayacucho-battle",
    "https://es.wikipedia.org/wiki/Capitulaci%C3%B3n_de_Ayacucho",
    "https://es.wikipedia.org/wiki/Santuario_hist%C3%B3rico_de_la_Pampa_de_Ayacucho",
    "https://elpais.com/america/2024-12-09/ayacucho-diciembre-9-1824-el-final-de-un-imperio-y-el-inicio-de-america-latina.html"
]

documentos = [extraer_texto(url) for url in urls]
print(f"✅ Se extrajeron {sum(1 for d in documentos if d)} documentos válidos.")

✅ Se extrajeron 9 documentos válidos.


In [4]:
# 1. Embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# 2. Chunking por oraciones
def dividir_en_chunks(texto, max_len=400):
    oraciones = re.split(r'(?<=[.?!])\s+', texto)
    chunks, actual = [], ""
    for oracion in oraciones:
        if len(actual) + len(oracion) <= max_len:
            actual += " " + oracion
        else:
            chunks.append(actual.strip())
            actual = oracion
    if actual:
        chunks.append(actual.strip())
    return chunks

# 3. Procesar todos los documentos
chunks = []
for doc in documentos:
    if doc:  # evita errores por documentos vacíos
        chunks.extend(dividir_en_chunks(doc))

# 4. Generar embeddings
vectores = embedding_model.encode(chunks)

# 5. Crear cliente Chroma
chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))

# Crear colección nueva
collection = chroma_client.get_or_create_collection("batalla_ayacucho_rag")

# 6. Cargar datos
for i, chunk in enumerate(chunks):
    collection.add(documents=[chunk], ids=[f"chunk_{i}"], embeddings=[vectores[i].tolist()])

print(f"✅ {len(chunks)} chunks embebidos y cargados en Chroma.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ 488 chunks embebidos y cargados en Chroma.


In [5]:
#Descarga de OLlama
!curl -fsSL https://ollama.com/install.sh | sh  # Solo una vez

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [6]:
#Lanzar servidor de OLlama en local
!ollama serve > /dev/null 2>&1 &
!sleep 10

In [7]:
#Descarga del modelo y librerías
!ollama pull phi3.5:latest
!pip install -U langchain-ollama

In [8]:
# Carga del modelo
from langchain_ollama import OllamaLLM
llm = OllamaLLM(
    model="phi3.5:latest",
    temperature=0.3,
    num_gpu=40,
    system=""
)

In [9]:
def responder_pregunta(pregunta: str, k=3, max_chars=1500) -> str:
    # Embedding de la pregunta
    query_emb = embedding_model.encode(pregunta).tolist()

    # Recuperar documentos similares
    resultados = collection.query(
        query_embeddings=[query_emb],
        n_results=k
    )

    # Construcción de contexto
    contextos = resultados["documents"][0]
    contexto = "\n\n".join(contextos)[:max_chars]

    # Prompt para Ollama + phi3.5
    prompt = f"""Responde en español a la siguiente pregunta sobre historia peruana.

  Contexto histórico:
  {contexto}

  Pregunta: {pregunta}
  Respuesta:"""

    # Llamada al modelo local
    return llm.invoke(prompt).strip()


In [13]:
pregunta = "¿Cuantas personas aproximandamente murieron en la batalla de Ayacucho?"
respuesta = responder_pregunta(pregunta)

print("🧠 Pregunta:", pregunta)
print("📘 Respuesta:", respuesta)

🧠 Pregunta: ¿Cuantas personas aproximandamente murieron en la batalla de Ayacucho?
📘 Respuesta: La batalla de Ayacucho, que tuvo lugar el 9 de diciembre de 1de 1824, es considerada una de las batallas decisivas para la independencia del Perú. Aunque no existe un dato exacto y universalmente aceptado sobre el número de bajas, generalmente se estima que entre 300 y 50 extranjeros (principalmente españoles) y aproximadamente 400-600 soldados peruanos murieron durante la contienda. Estas cifras pueden variar según las distintas fuentes históricas, pero reflejan el significativo costo humano de esta batalla que selló definitivamente el destino independentista del Perú y marcó un punto irreversible en América Latina contra el colonialismo español.

Para una respuesta más precisa se recomienda consultar estudios especializados o fuentes históricas confiables, ya que los registros exactos pueden haber sido difíciles de determinar debido a las circunstancias y la naturaleza del conflicto. La es